# Creation of the CNN

### Importations 

In [1]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import pickle as pkl
import pandas as pd


from torch.autograd import Variable
from torch import nn, optim
from torch.nn import functional as F
from matplotlib import gridspec




print(torch.__version__)

1.0.1.post2


### The Aim 

Some explation about the purpose of this project.

### The Data

Some insight about the data.

In [2]:
Dataset = pd.read_pickle("/home/janjar/Dataset/Trainingset/Training_dataframe.pkl")
Dataset.loc[0:3]

,Room,Array_position,Distance,Absorption,SNR,Audio_file,Phase_Matrix,Label
0,0,"[3.395545610109257, 2.0550302704519794, 1.5]",1.0,0.702471,0,/home/janjar/Dataset/Trainingset/audio_signals...,/home/janjar/Dataset/Trainingset/phase_matrix/...,21
1,0,"[3.2402107165724896, 1.8276250338752629, 1.5]",1.0,0.702471,0,/home/janjar/Dataset/Trainingset/audio_signals...,/home/janjar/Dataset/Trainingset/phase_matrix/...,32
2,0,"[1.6801067878855174, 1.927483519251545, 1.5]",1.0,0.702471,0,/home/janjar/Dataset/Trainingset/audio_signals...,/home/janjar/Dataset/Trainingset/phase_matrix/...,9
3,0,"[3.2065269137273935, 3.207686173511143, 1.5]",1.0,0.702471,0,/home/janjar/Dataset/Trainingset/audio_signals...,/home/janjar/Dataset/Trainingset/phase_matrix/...,27


#### Defining our number of samples.

In [3]:
number_of_rooms = Dataset.shape[0]

### Loading the phase matrices in tensors with respect to the indexing shown in the dataframe.

In [4]:
#path containing the Training data on the machine you are working on.
#path = "/home/janjar/Dataset/Trainingset/"

data_matrix = np.zeros([number_of_rooms,4,129,390])
for i in range(number_of_rooms):
    fileName_matrix = Dataset.iloc[i]['Phase_Matrix']
    fileObject2 = open(fileName_matrix, 'rb')
    matrix_loaded = pkl.load(fileObject2)
    fileObject2.close()
    data_matrix[i] = matrix_loaded
data_matrix.shape    

(140, 4, 129, 390)

In [5]:
data_matrix = torch.from_numpy(data_matrix)
data_matrix = data_matrix.view(-1,4,129,390)
data_matrix = data_matrix.type('torch.FloatTensor')
data_matrix.shape

torch.Size([140, 4, 129, 390])

In [6]:
data_matrix[127]

tensor([[[ 3.1416,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  3.1416],
         [ 0.0720,  1.3791, -3.0578,  ...,  2.6336, -2.4902,  2.2125],
         [-3.0836, -1.9785, -0.7836,  ...,  2.2236,  0.8599,  1.3155],
         ...,
         [ 2.3806,  0.5016,  1.5734,  ...,  2.0116,  1.1742,  1.3672],
         [-0.4633,  1.1377, -1.4635,  ..., -0.2904, -1.7364,  0.6829],
         [ 3.1416,  3.1416,  3.1416,  ...,  3.1416,  3.1416,  0.0000]],

        [[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  3.1416,  0.0000],
         [-2.5534,  1.0936,  2.9456,  ...,  2.9508,  0.1252, -1.0486],
         [ 0.2845,  1.2998, -0.9096,  ..., -1.2360, -2.9721, -1.9876],
         ...,
         [-0.5019, -1.2472,  1.9196,  ..., -2.8629, -2.9445, -1.9050],
         [ 1.5748,  2.7410, -0.5381,  ...,  0.3929, -0.1214, -2.5252],
         [ 3.1416,  0.0000,  3.1416,  ...,  3.1416,  3.1416,  3.1416]],

        [[ 0.0000,  3.1416,  0.0000,  ...,  3.1416,  3.1416,  0.0000],
         [ 1.8699,  0.4906,  3.0916,  ..., -0

### Function that reshape a label into the right format for the data_targets.

In [7]:
def prep_targets(index,label):
    target = torch.tensor(label)
    target = target.expand(390,1)
    return target

In [8]:
data_targets = torch.zeros([number_of_rooms], dtype=torch.float64)
for i in range(number_of_rooms):
    data_targets[i] = Dataset.iloc[i]['Label']
data_targets.shape   

torch.Size([140])

# Architecture 1.0 : vanilla CNN for DOA 

In this section, we will focus on building a vanilla CNN in order to recognize the directions of arrival of the sound in a specific room. Once done, we will then compare "by hand" the labelel dataset to the predicted values. This is the most basic setup and will try to improve latter.

## 1.1 Creating Model

In [27]:
class Net(nn.Module):
    def __init__(self,nb_hidden = 512):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, kernel_size=2)
        self.conv2 = nn.Conv2d(64, 64, kernel_size=2)
        self.conv3 = nn.Conv2d(64 , 64, kernel_size=2)
        self.fc1 = nn.Linear(3120768, nb_hidden)# (1x8064) being the dim of the censor obtained by flattening the output of the 3rd CL.
        self.fc2 = nn.Linear(nb_hidden*4,nb_hidden)
        self.fc3 = nn.Linear(nb_hidden,37)
        
    
    def forward(self, x):
        x_0 = x[:,:,0,:,:].view(-1,1,129,390)
        x_1 = x[:,:,1,:,:].view(-1,1,129,390)
        x_2 = x[:,:,2,:,:].view(-1,1,129,390)
        x_3 = x[:,:,3,:,:].view(-1,1,129,390)

        x_0 = F.relu(self.conv1(x_0))         
        x_1 = F.relu(self.conv1(x_1))
        x_2 = F.relu(self.conv1(x_2))
        x_3 = F.relu(self.conv1(x_3))

        x_0 = F.relu(self.conv2(x_0))
        x_1 = F.relu(self.conv2(x_1))
        x_2 = F.relu(self.conv2(x_2))
        x_3 = F.relu(self.conv2(x_3))
        
        x_0 = F.relu(self.conv3(x_0))
        x_1 = F.relu(self.conv3(x_1))
        x_2 = F.relu(self.conv3(x_2))
        x_3 = F.relu(self.conv3(x_3))            

        x_0 = F.relu(self.fc1(x_0.view(-1, 3120768)))
        x_1 = F.relu(self.fc1(x_1.view(-1, 3120768)))
        x_2 = F.relu(self.fc1(x_2.view(-1, 3120768)))            
        x_3 = F.relu(self.fc1(x_3.view(-1, 3120768))) 

        x = torch.cat((x_0,x_1,x_2,x_3),1)            

        x = F.relu(self.fc2(x))

        x = F.relu(self.fc3(x))

        return x            

In [ ]:

    def prep_to_forwad(x):
        out = torch.zeros([number_of_rooms], dtype=torch.float64)
        for i in range(390):
            out[i] = x[:,:,:,i,:]
    
    def forward(self, x):
        
        

### Preparing the Data

In [47]:
def prep_input_vanilla(train_input):
    new_train_input = train_input.view(-1,1,4,129,390)
    return new_train_input

In [48]:
train_data = prep_input_vanilla(data_matrix)
train_data.shape

torch.Size([140, 1, 4, 129, 390])

In [49]:
def prep_labels_vanilla(train_input):
    new_train_input = train_input.view(-1)
    return new_train_input

In [50]:
train_targets = prep_labels_vanilla(data_targets)
train_targets = train_targets.to(dtype=torch.int64)
train_data, train_targets = Variable(train_data), Variable(train_targets)
train_data.shape

torch.Size([140, 1, 4, 129, 390])

In [63]:
train_data = train_data[0:50]
train_targets = train_targets[0:50]
train_data.shape


torch.Size([20, 1, 4, 129, 390])

### End of Sanity check.

## 1.2 Training the model

In [52]:
model = Net()

In [69]:
mini_batch_size = 10 
nb_epochs = 10
eta = 0.5 #learning rate
#criterion = nn.MSELoss() # MeanSquaredloss
criterion = torch.nn.CrossEntropyLoss() #Cross Entropy
optimizer = torch.optim.SGD(model.parameters(), lr = eta, momentum = 0.95) #Stochastic gradient descent

### Need to figure out how to use mini-batches to gain time in this configuration.
#### train_model is the method used to update the weights.

In [70]:
def train_model(model, train_input, train_target, mini_batch_size, criterion, nb_epochs, eta):
    
    optimizer = torch.optim.SGD(model.parameters(), lr = eta, momentum = 0.95) #Stochastic gradient descent
    
    for e in range(0, nb_epochs):
        # We do this with mini-batches
        for b in range(0, train_input.size(0)):
            for a in range(0,train_input.size(1)):
                output = model(train_input[b,a,:,:,:,:])
                target = train_target[b,a,:]
                print("Time : Output vs target",a,output,target)
                #loss = criterion(output,target)
                loss = criterion(output,target)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
            
                print(e, loss.data.item())
    

In [72]:
def train_model_3(model, optimizer, nb_epochs, train_input, train_target ,mini_batch_size):

    #start = time.time()
    for e in range(0,nb_epochs):
        for b in range(0, train_input.size(0), mini_batch_size):
            output = model(train_input.narrow(0, b, mini_batch_size))
            target = train_target.narrow(0, b, mini_batch_size)
            print("Time : Output vs target",b,output,target)
            loss = criterion(output,target)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
    #end = time.time()

        print(e,loss.data.item())

    

In [73]:
train_model_3(model,optimizer,nb_epochs,train_data,train_targets,mini_batch_size)

Time : Output vs target 0 tensor([[0.0000, 1.1370, 0.0000, 0.0000, 1.0357, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 1.1360, 0.0000, 0.0000, 0.0000, 1.2602,
         0.0000, 0.0000, 0.0000, 1.4026, 0.0000, 1.1433, 1.0351, 0.0000, 0.0000,
         1.0382, 0.0000, 0.0000, 0.0000, 0.0000, 1.0225, 0.0000, 0.0000, 1.2304,
         1.0695],
        [0.0000, 1.1370, 0.0000, 0.0000, 1.0357, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 1.1360, 0.0000, 0.0000, 0.0000, 1.2602,
         0.0000, 0.0000, 0.0000, 1.4026, 0.0000, 1.1433, 1.0351, 0.0000, 0.0000,
         1.0382, 0.0000, 0.0000, 0.0000, 0.0000, 1.0225, 0.0000, 0.0000, 1.2304,
         1.0695],
        [0.0000, 1.1370, 0.0000, 0.0000, 1.0357, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 1.1360, 0.0000, 0.0000, 0.0000, 1.2602,
         0.0000, 0.0000, 0.0000, 1.4026, 0.0000, 1.1433, 1.0351, 0.0000, 0.0000,
         1.0382, 0.0000, 0.0000, 0.0000, 0.0000

Time : Output vs target 10 tensor([[0.0000, 0.6876, 0.0000, 0.0000, 0.9049, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.6882, 0.0000, 0.0000, 0.0000, 0.7124,
         0.0000, 0.0000, 0.0000, 1.2422, 0.0000, 0.6874, 0.9041, 0.0000, 0.0000,
         1.0607, 0.0000, 0.0000, 0.0000, 0.0000, 1.0401, 0.0000, 0.0000, 0.7125,
         0.9128],
        [0.0000, 0.6876, 0.0000, 0.0000, 0.9049, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.6882, 0.0000, 0.0000, 0.0000, 0.7124,
         0.0000, 0.0000, 0.0000, 1.2422, 0.0000, 0.6874, 0.9041, 0.0000, 0.0000,
         1.0607, 0.0000, 0.0000, 0.0000, 0.0000, 1.0401, 0.0000, 0.0000, 0.7125,
         0.9128],
        [0.0000, 0.6876, 0.0000, 0.0000, 0.9049, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.6882, 0.0000, 0.0000, 0.0000, 0.7124,
         0.0000, 0.0000, 0.0000, 1.2422, 0.0000, 0.6874, 0.9041, 0.0000, 0.0000,
         1.0607, 0.0000, 0.0000, 0.0000, 0.000

2 3.3507862091064453
Time : Output vs target 0 tensor([[0.0000, 0.8475, 0.0000, 0.0000, 0.8995, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.8474, 0.0000, 0.0000, 0.0000, 0.8417,
         0.0000, 0.0000, 0.0000, 1.0911, 0.0000, 0.8446, 0.8976, 0.0000, 0.0000,
         1.0949, 0.0000, 0.0000, 0.0000, 0.0000, 1.0882, 0.0000, 0.0000, 0.8511,
         0.9001],
        [0.0000, 0.8475, 0.0000, 0.0000, 0.8995, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.8474, 0.0000, 0.0000, 0.0000, 0.8417,
         0.0000, 0.0000, 0.0000, 1.0911, 0.0000, 0.8446, 0.8976, 0.0000, 0.0000,
         1.0949, 0.0000, 0.0000, 0.0000, 0.0000, 1.0882, 0.0000, 0.0000, 0.8511,
         0.9001],
        [0.0000, 0.8475, 0.0000, 0.0000, 0.8995, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.8474, 0.0000, 0.0000, 0.0000, 0.8417,
         0.0000, 0.0000, 0.0000, 1.0911, 0.0000, 0.8446, 0.8976, 0.0000, 0.0000,
         1.0949, 0.0000, 0

Time : Output vs target 10 tensor([[0.0000, 1.0423, 0.0000, 0.0000, 0.8800, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 1.0415, 0.0000, 0.0000, 0.0000, 1.0187,
         0.0000, 0.0000, 0.0000, 0.9824, 0.0000, 1.0383, 0.8775, 0.0000, 0.0000,
         1.1435, 0.0000, 0.0000, 0.0000, 0.0000, 1.1529, 0.0000, 0.0000, 1.0320,
         0.8762],
        [0.0000, 1.0423, 0.0000, 0.0000, 0.8800, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 1.0415, 0.0000, 0.0000, 0.0000, 1.0187,
         0.0000, 0.0000, 0.0000, 0.9824, 0.0000, 1.0383, 0.8775, 0.0000, 0.0000,
         1.1435, 0.0000, 0.0000, 0.0000, 0.0000, 1.1529, 0.0000, 0.0000, 1.0320,
         0.8762],
        [0.0000, 1.0423, 0.0000, 0.0000, 0.8800, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 1.0415, 0.0000, 0.0000, 0.0000, 1.0187,
         0.0000, 0.0000, 0.0000, 0.9824, 0.0000, 1.0383, 0.8775, 0.0000, 0.0000,
         1.1435, 0.0000, 0.0000, 0.0000, 0.000

5 3.209578275680542
Time : Output vs target 0 tensor([[0.0000, 1.3128, 0.0000, 0.0000, 0.9639, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 1.3107, 0.0000, 0.0000, 0.0000, 1.2940,
         0.0000, 0.0000, 0.0000, 0.9092, 0.0000, 1.3105, 0.9617, 0.0000, 0.0000,
         1.1381, 0.0000, 0.0000, 0.0000, 0.0000, 1.1609, 0.0000, 0.0000, 1.3007,
         0.9602],
        [0.0000, 1.3128, 0.0000, 0.0000, 0.9639, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 1.3107, 0.0000, 0.0000, 0.0000, 1.2940,
         0.0000, 0.0000, 0.0000, 0.9092, 0.0000, 1.3105, 0.9617, 0.0000, 0.0000,
         1.1381, 0.0000, 0.0000, 0.0000, 0.0000, 1.1609, 0.0000, 0.0000, 1.3007,
         0.9602],
        [0.0000, 1.3128, 0.0000, 0.0000, 0.9639, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 1.3107, 0.0000, 0.0000, 0.0000, 1.2940,
         0.0000, 0.0000, 0.0000, 0.9092, 0.0000, 1.3105, 0.9617, 0.0000, 0.0000,
         1.1381, 0.0000, 0.

Time : Output vs target 10 tensor([[0.0000, 1.0536, 0.0000, 0.0000, 1.0087, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 1.0522, 0.0000, 0.0000, 0.0000, 0.9956,
         0.0000, 0.0000, 0.0000, 0.8415, 0.0000, 1.0506, 1.0071, 0.0000, 0.0000,
         1.0583, 0.0000, 0.0000, 0.0000, 0.0000, 1.0713, 0.0000, 0.0000, 1.0113,
         0.9917],
        [0.0000, 1.0536, 0.0000, 0.0000, 1.0087, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 1.0522, 0.0000, 0.0000, 0.0000, 0.9956,
         0.0000, 0.0000, 0.0000, 0.8415, 0.0000, 1.0506, 1.0071, 0.0000, 0.0000,
         1.0583, 0.0000, 0.0000, 0.0000, 0.0000, 1.0713, 0.0000, 0.0000, 1.0113,
         0.9917],
        [0.0000, 1.0536, 0.0000, 0.0000, 1.0087, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 1.0522, 0.0000, 0.0000, 0.0000, 0.9956,
         0.0000, 0.0000, 0.0000, 0.8415, 0.0000, 1.0506, 1.0071, 0.0000, 0.0000,
         1.0583, 0.0000, 0.0000, 0.0000, 0.000

8 3.2376129627227783
Time : Output vs target 0 tensor([[0.0000, 0.9611, 0.0000, 0.0000, 1.1039, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.9604, 0.0000, 0.0000, 0.0000, 0.9070,
         0.0000, 0.0000, 0.0000, 0.8536, 0.0000, 0.9602, 1.1037, 0.0000, 0.0000,
         0.9830, 0.0000, 0.0000, 0.0000, 0.0000, 0.9869, 0.0000, 0.0000, 0.9184,
         1.0858],
        [0.0000, 0.9611, 0.0000, 0.0000, 1.1039, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.9604, 0.0000, 0.0000, 0.0000, 0.9070,
         0.0000, 0.0000, 0.0000, 0.8536, 0.0000, 0.9602, 1.1037, 0.0000, 0.0000,
         0.9830, 0.0000, 0.0000, 0.0000, 0.0000, 0.9869, 0.0000, 0.0000, 0.9184,
         1.0858],
        [0.0000, 0.9611, 0.0000, 0.0000, 1.1039, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.9604, 0.0000, 0.0000, 0.0000, 0.9070,
         0.0000, 0.0000, 0.0000, 0.8536, 0.0000, 0.9602, 1.1037, 0.0000, 0.0000,
         0.9830, 0.0000, 0